In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

live_config_files = glob.glob('cfgs_live/*', recursive=True)
symbols = []
symbol_files = {}
for f in live_config_files:
    symbol = f.split('\\')[-1]
    symbol = symbol.replace('.json', 'USDT')
    symbols.append(symbol)
    symbol_files[symbol] = f
# symbol_files
len(symbols)

15

In [3]:
# start_files = ['start_long_only.sh', 'start_long_short.sh']
# long_symbols = []
# short_symbols = []
# for filename in start_files:
#     with open(filename, 'r') as file:
#         lines = file.read().split('\n')
#         for l in lines:
#             if 'screen -dmS ' in l:
#                 symbol = l.strip().split(" ")[-1]
#                 long_symbols.append(symbol) if filename == 'start_long_only.sh' else short_symbols.append(symbol)
#
# common_in_both = list(set(long_symbols).intersection(short_symbols))
# print(f'common_in_both : {common_in_both}')
#
# seen = set()
# long_dups = [x for x in long_symbols if x in seen or seen.add(x)]
# print(f'long_dups : {long_dups}')
# seen = set()
# short_dups = [x for x in short_symbols if x in seen or seen.add(x)]
# print(f'short_dups : {short_dups}')
#
# print(f'total: {len(symbols)}')
# print(f'long: {len(long_symbols)}')
# print(f'short: {len(short_symbols)}')
#
# if len(long_symbols) + len(short_symbols) != len(symbols):
#     print('Lengths DONT MATCH!')
# else:
#     print(f'No duplicate found')

In [4]:
import pandas as pd
config = pd.read_csv('run_config.csv')
#shorts = config[(config['short_mode'] == 'p')]
short_exposure = config.loc[config['short_mode'] == 'n', 'short_exposure'].sum()
long_exposure = config.loc[config['long_mode'] == 'n', 'long_exposure'].sum()

total = short_exposure + long_exposure
print(f'short_exposure: {short_exposure}, long_exposure:{long_exposure}, total:{total}')

short_exposure: 0.0, long_exposure:0.0, total:0.0


In [5]:

config = pd.read_csv('run_config.csv')
#config.loc[config['short_mode'] == 'n', 'short_exposure'] = 0.025
#config.loc[config['long_mode'] == 'n', 'long_exposure'] = 0.03
#
# short_exposure = config.loc[config['short_mode'] == 'n', 'short_exposure'].sum()
# long_exposure = config.loc[config['long_mode'] == 'n', 'long_exposure'].sum()
#
# total = short_exposure + long_exposure
# print(f'short_exposure: {short_exposure}, long_exposure:{long_exposure}, total:{total}')

#config = pd.DataFrame()

# for symbol in long_symbols:
#     config = config.append({
#         'symbol':symbol,
#         'direction':'long',
#         'long_mode':'n',
#         'short_mode':'p',
#         'long_exposure':0.035,
#         'short_exposure':0.0
#     }, ignore_index=True)
#
# for symbol in short_symbols:
#     config = config.append({
#         'symbol':symbol,
#         'direction':'long_short',
#         'long_mode':'n',
#         'short_mode':'n',
#         'long_exposure':0.035,
#         'short_exposure':0.02
#     }, ignore_index=True)
#config.loc[config['short_mode'] == 'p', 'short_mode'] = 'm'
#config.loc[config['long_mode'] == 'p', 'long_mode'] = 'm'
#config.to_csv('run_config.csv')
config

,Unnamed: 0,symbol,direction,long_mode,short_mode,long_exposure,short_exposure,account
0,4,EOS,long_short,gs,gs,0.07,0.04,binance_01
1,6,LINK,long_short,gs,gs,0.07,0.04,binance_01
2,12,XTZ,long_short,gs,gs,0.07,0.04,binance_01


In [6]:
# #### DCA Helper
# z_dca_long = pd.read_csv('0dca.csv')
# z_dca_long = z_dca_long[(z_dca_long['Exposure'] >= 0.04)]
#
# z_dca_long_symbols = z_dca_long['symbol'].tolist()
# z_dca_long_symbols = [w.replace('USDT','') for w in z_dca_long_symbols]
# print(z_dca_long_symbols)
# config.loc[config.symbol.isin(z_dca_long_symbols), 'long_mode'] = 'm'
# #config.to_csv('run_config.csv')
config

,Unnamed: 0,symbol,direction,long_mode,short_mode,long_exposure,short_exposure,account
0,4,EOS,long_short,gs,gs,0.07,0.04,binance_01
1,6,LINK,long_short,gs,gs,0.07,0.04,binance_01
2,12,XTZ,long_short,gs,gs,0.07,0.04,binance_01


In [7]:
import os
import shutil

with open('run_template.sh') as f:
    run_template = f.read()

start_template = 'screen -X -S {symbol}_binance_v5_9_0 kill\n' \
                 'chmod +x cfgs_runtime/{symbol}.sh\n' \
                 'screen -dmS {symbol}_binance_v5_9_0 ./cfgs_runtime/{symbol}.sh {symbol}'
start_commands =[]
for index, row in config.iterrows():
    dir = f'cfgs_runtime/'
    run_command = f'-sm {row["short_mode"]} -sw {row["short_exposure"]} -lm {row["long_mode"]} -lw {row["long_exposure"]}'
    full_run_command = run_template.format(command = run_command)
    #print(full_run_command)
    with open(f'{dir}/{row["symbol"]}.sh','w') as writer:
        writer.write(full_run_command)

    start_commands.append(start_template.format(symbol = row["symbol"]))

full_start_command = "\n\n".join(map(str,start_commands))
print(full_start_command)
with open(f'start.sh','w') as writer:
        writer.write(full_start_command)

screen -X -S EOS_binance_v5_9_0 kill
chmod +x cfgs_runtime/EOS.sh
screen -dmS EOS_binance_v5_9_0 ./cfgs_runtime/EOS.sh EOS

screen -X -S LINK_binance_v5_9_0 kill
chmod +x cfgs_runtime/LINK.sh
screen -dmS LINK_binance_v5_9_0 ./cfgs_runtime/LINK.sh LINK

screen -X -S XTZ_binance_v5_9_0 kill
chmod +x cfgs_runtime/XTZ.sh
screen -dmS XTZ_binance_v5_9_0 ./cfgs_runtime/XTZ.sh XTZ


In [8]:
total_wallet = 607
margin = 7
coins = config.shape[0]
longs = config[(config['long_mode'] == 'n')].shape[0]
shorts = config[(config['short_mode'] == 'n')].shape[0]

ratio = shorts/longs

total_exposure = longs+shorts
total_wallet_with_margin = total_wallet * margin
per_entry = total_wallet_with_margin / total_exposure
money_cost = total_wallet / total_exposure

exposure = money_cost / margin

xxx = (exposure / total_wallet) * 100

print(f'coins: {coins}, longs:{longs}, shorts:{shorts} , ratio:{ratio}')
xxx

ZeroDivisionError: division by zero